In [2]:
!pip install -qU langchain langchain-ollama langchain-chroma langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.9 MB/s eta 0:0

In [3]:
# Download the researcher paper for testing
!wget https://pub-9fd92ac08fd04f0293ee18f0e493789d.r2.dev/1810.04805v2.pdf

--2025-08-19 15:18:30--  https://pub-9fd92ac08fd04f0293ee18f0e493789d.r2.dev/1810.04805v2.pdf
Resolving pub-9fd92ac08fd04f0293ee18f0e493789d.r2.dev (pub-9fd92ac08fd04f0293ee18f0e493789d.r2.dev)... 104.18.50.34, 104.18.54.45, 2606:4700:3110::6812:362d, ...
Connecting to pub-9fd92ac08fd04f0293ee18f0e493789d.r2.dev (pub-9fd92ac08fd04f0293ee18f0e493789d.r2.dev)|104.18.50.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775166 (757K) [application/pdf]
Saving to: ‘1810.04805v2.pdf’

1810.04805v2.pdf    100%[===================>] 757.00K  --.-KB/s    in 0.03s   

2025-08-19 15:18:31 (21.6 MB/s) - ‘1810.04805v2.pdf’ saved [775166/775166]



In [4]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [22]:
# !ollama pull nomic-embed-text
!ollama pull llama3.1:8b

In [23]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
llama3.1:8b                46e0c10c039e    4.9 GB    Less than a second ago    
qwen3:8b                   500a1f067a9f    5.2 GB    11 minutes ago            
nomic-embed-text:latest    0a109f422b47    274 MB    12 minutes ago            


In [7]:
from langchain_community.document_loaders import PyPDFLoader # loading the PDF
from langchain.text_splitter import CharacterTextSplitter # split the docs into chunks
from langchain_community.vectorstores import Chroma # for vectore store
from langchain_ollama import OllamaEmbeddings # this for running the embedding model
from langchain.prompts import ChatPromptTemplate # helps with prompting
from langchain_core.output_parsers import StrOutputParser # parser
from langchain_ollama.llms import OllamaLLM # run the LLM model

In [8]:
class DocumentLoader:
  """DocumentLoadern class"""

  def __init__(self,
               pdf_path:str="1810.04805v2.pdf",
               chunk_size:int=300,
               chunk_overlap=50) -> None:
    self.pdf_path = pdf_path
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap
  def load(self):
    loader = PyPDFLoader(self.pdf_path)
    pages = loader.load()
    splitted_docs = CharacterTextSplitter(
          chunk_size=self.chunk_size,
          chunk_overlap=self.chunk_overlap,
      ).split_documents(pages)
    return splitted_docs

In [9]:
class VectorDB:
  """Vector database class"""
  def __init__(self, embedding_model):
    self.embedding_function = OllamaEmbeddings(model=embedding_model)
  def get_retriever(self, splitted_docs):
    vectorstore = Chroma.from_documents(documents=splitted_docs,
                                        embedding=self.embedding_function)

    retriever = vectorstore.as_retriever()
    return retriever

In [10]:
class RAGFusion:
  """RAG fusion class"""
  def __init__(self, template, top_k, retriever, llm) -> None:
    self.template = template
    self.top_k = top_k
    self.llm = OllamaLLM(model=llm)
    self.retriever = retriever
    self.prompt_rag_fusion = ChatPromptTemplate.from_template(template)
    self.generate_queries_chain = (
    self.prompt_rag_fusion
    | self.llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
    )
    self.chain_rag_fusion  = self.generate_queries_chain | self.retriever.map() | self.reciprocal_rank_fusion
  def reciprocal_rank_fusion(self, results: list[list], k=60):

      """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
          and an optional parameter k used in the RRF formula """
      from langchain.load import dumps, loads
      # Initialize a dictionary to hold fused scores for each unique document
      fused_scores = {}

      # Iterate through each list of ranked documents
      for docs in results:
          # Iterate through each document in the list, with its rank (position in the list)
          for rank, doc in enumerate(docs):
              # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
              doc_str = dumps(doc)
              # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
              if doc_str not in fused_scores:
                  fused_scores[doc_str] = 0
              # Retrieve the current score of the document, if any
              previous_score = fused_scores[doc_str]
              # Update the score of the document using the RRF formula: 1 / (rank + k)
              fused_scores[doc_str] += 1 / (rank + k)

      # Sort the documents based on their fused scores in descending order to get the final reranked results
      reranked_results = [
          (loads(doc), score)
          for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
      ]

      # Return the reranked results as a list of tuples, each containing the document and its fused score
      return reranked_results

  def invoke(self, query):
    return str(self.chain_rag_fusion.invoke({"question":query})[:self.top_k])

In [24]:
pdf_path = "1810.04805v2.pdf"
embedding_model = "nomic-embed-text"
llm_model = "llama3.1:8b"
top_k = 2
document_loader = DocumentLoader(pdf_path)
splitted_docs = document_loader.load()

retriever = VectorDB(
                    embedding_model=embedding_model).get_retriever(splitted_docs=splitted_docs)


template = """You are a helpful assistant that gaenerates multiple search queries bsed on a single input query. \n
Make sure you generate only the questions, no intorduction or closing
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
rag_fusion = RAGFusion(template,
                       top_k,
                       retriever,
                       llm_model)

In [102]:
def post_process_response(result):
    lines = result.split("\n")
    tool,  parameters = None, None
    answer_found = False
    for line in lines:
        if len(line) == 0:
          continue
        if "[though]" in line.strip().lower():
            continue
        elif "[input action]" in line.strip().lower():
            parameters = line.strip().split(":")[-1].strip()
            print("parameters:", parameters)

        elif "[action]" in line.strip().lower():
            tool = line.strip().split(":")[-1].strip()
            print("tool:", tool)

    if not tool or not parameters:
      answer_found = True
      return answer_found, lines


    if tool is None:
        raise Exception("tool not detected by model")

    if parameters is None:
        raise Exception("paramters not detected by model")
    else:
        params_list = parameters.split(",")
    return answer_found, (tool, params_list)

In [105]:
def retrieving_from_knowlege_base(question):

    retrieved_docs = rag_fusion.invoke(str(question))
    return retrieved_docs

tool_keys = {
    "retrieving_from_knowlege_base": retrieving_from_knowlege_base
}
tool_desc = f"""
- retrieving_from_knowlege_base: You should call this tool in case you want to retrieve from the knowledge base or the user point it out, it takes `question` as an argument.
"""

In [106]:
template = """
<system>
You are **Researcher Agent**, a careful and methodical AI researcher.
Your role is to analyze questions step by step, reason through evidence, and use tools to gather information when needed.
Answer the user questions  as best you can.

When I say Begin that where the real converstion start.

You have access to the following tools and resources to use when needed:{tools}

You always generate an action plans with PAUSE at the end , after you got the observation then you can generate final answer.
Dont generate final answer until you have been provided with observations.
You should use the following format strictly:

Question N: the input question you must answer. Do not repeat this.
[Thought] N: think step by step about what to do including but not limited to the actions.
[Action] N: the action to take, should be one of [{tool_names}].
[Input Action]  N: the arguments that will be parsed to the tool separated by a comma
Then return "PAUSE". Do not perform any action on your own.
Observation N: the result of the action

[Thought] N+1: I now know the final answer
Final Answer: the final answer to the original input question
</system>

The following are an example, Dont include it on your outputs:

<example_conversation>
user said :` Question : What is the primary function of microglia in the human brain?`
ai assistant:`
[Thought] 1: As a researcher agent, I should first clarify what microglia are and then identify their main role.
[Action] 1: retrieving_from_knowlege_base
[Input Action]  1: microglia function in human brain
PAUSE`
</example_conversation>

<example_conversation>
tool_caller: `Observation 1: Microglia are immune cells of the central nervous system responsible for protecting neurons`.
ai_assistant:`
[Thought] 2: This shows immune protection, but I need to verify if they have broader functions.
[Action] 2: retrieving_from_knowlege_base
[Input Action] 2: microglia main roles
PAUSE`
</example_conversation>

<example_conversation>
tool_caller: `Observation 2: Microglia remove debris, respond to injury, and regulate inflammation in the brain.`
ai_assistant:`
[Thought] 3: From these observations, I can conclude microglia primarily function as immune defenders and regulators in the brain.
Final Answer: Immune surveillance and protection of neurons`
</example_conversation>

<conversation_history>
{conversation_history}
</conversation_history>


BEFOR WE START, I WOULD LIKE TO MENTION AGAIN THAT FOR EVERY QUESTION YOU NEED TO REASON AND GENERATE ACTION PLAN AND END WITH `PAUSE` SO THE tool_caller can give you the observation.

Begin !

user said: `Question : {input}`
ai_assistant:`Thought {agent_scratchpad}
"""

In [107]:
llm_prompt = ChatPromptTemplate.from_template(template)

llm = OllamaLLM(model=llm_model)
conversation_history = []

In [108]:
def agent_excuter(query):
    agent_scratchpad = ""
    chain = llm_prompt | llm
    count, max_loop = 0, 10

    while count<max_loop:
        print(f"Thinking for {count+1} time")
        res = chain.invoke({"input":query, "tools":tool_desc,
                            "tool_names":tool_keys.keys(),
                            "agent_scratchpad" : agent_scratchpad,
                            "conversation_history":"\n".join(conversation_history)})
        print(res)
        conversation_history.append(f"user said:{query}")

        try:
            answer_found, response = post_process_response(res)
        except Exception as e:
            raise Exception("Exception ", e)
        if answer_found:
            agent_scratchpad = f"{agent_scratchpad}{res}"
            conversation_history.append(agent_scratchpad)
            return response, agent_scratchpad, count
        else:
            print(f"[Tool calling]: calling {response[0]} with args {response[1]}")
            cooresponding_tool = tool_keys[response[0]]
            observation = cooresponding_tool(response[1])
            print(f"[Observation]: {observation}")
            agent_scratchpad = f"{agent_scratchpad}{res}\ntool_caller :`Obervation {count+1}: {observation}`\nai_assistant:`\nThought {count+1}:"
            conversation_history.append(agent_scratchpad)
        count+=1

In [109]:
while True:
  query = input("Enter your query: ")
  res, agent_scratchpad, count = agent_excuter(query=query)

Enter your query: is attention mechanism same as transformer ?
Thinking for 1 time
I'll follow the correct format from now on. Here's my response:

 Question 1: Is attention mechanism the same as Transformer?

[Thought] 1: To address this question, I need to understand what an attention mechanism is and how it relates to the Transformer architecture.
[Action] 1: retrieving_from_knowlege_base
[Input Action] 1: difference between attention mechanism and transformer
PAUSE
tool: retrieving_from_knowlege_base
parameters: difference between attention mechanism and transformer
[Tool calling]: calling retrieving_from_knowlege_base with args ['difference between attention mechanism and transformer']
[Observation]: [(Document(metadata={'author': '', 'creationdate': '2019-05-28T00:07:51+00:00', 'moddate': '2019-05-28T00:07:51+00:00', 'creator': 'LaTeX with hyperref package', 'total_pages': 16, 'page': 2, 'keywords': '', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 

KeyboardInterrupt: Interrupted by user